In [1]:
import pandas as pd
import psycopg2, os
import matplotlib.pyplot as plt

# Establish connection and create its cursor
try: 
    conn = psycopg2.connect(f"host={os.environ['AURORA_POSTGRES_HOST']} dbname={os.environ['AURORA_POSTGRES_DATABASE']} user={os.environ['AURORA_POSTGRES_USERNAME']} password={os.environ['AURORA_POSTGRES_PWD']}")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    

Load tables

In [2]:
# Collect trips and events tables
cur.execute("SELECT * FROM zendrive.trip")
trips = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
cur.execute("SELECT * FROM zendrive.event")
events = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])

# Pick one example trip to look up
example_trip = trips[(trips['score'].between(10, 40)) & (trips['distance'].between(5, 10))].iloc[0]
display(example_trip)

# Collect its path
cur.execute(f"SELECT * FROM zendrive.path_point WHERE trip_id={example_trip.trip_id}")
trip_paths = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
print('Total of available trip points:', trip_paths.shape[0])
trip_paths.head()



id                                                       458
trip_id                                        1621634402930
driver                                                     7
created_at                        2021-05-27 09:30:19.683158
updated_at                        2021-06-18 14:00:07.170079
started_at                               2021-05-21 22:00:02
ended_at                                 2021-05-21 22:10:19
distance                                               7.369
score                                                   36.0
user_feedback                                           None
duration_seconds                                     616.329
trip_max_speed_kmph                                93.995735
latitude_start                                    -23.518255
longitude_start                                   -46.617634
latitude_end                                      -23.561278
longitude_end                                     -46.651066
hard_turn_rating        

Total of available trip points: 71


,trip_internal_id,trip_id,point_order_id,latitude,longitude,point_timestamp,time_millis,trip_created_at,trip_updated_at,point_composed_id,internal_timestamp
0,458,1621634402930,1,-23.518255,-46.617634,2021-05-21 19:00:02.930,1621634402930,2021-05-27 09:30:19.683158,2021-06-18 14:00:07.170079,458--1--2021-06-18 14:00:07.170079,2021-10-04 15:54:12.020790
1,458,1621634402930,2,-23.518238,-46.617756,2021-05-21 19:00:03.925,1621634403925,2021-05-27 09:30:19.683158,2021-06-18 14:00:07.170079,458--2--2021-06-18 14:00:07.170079,2021-10-04 15:54:12.020790
2,458,1621634402930,3,-23.518159,-46.618510,2021-05-21 19:00:07.000,1621634407929,2021-05-27 09:30:19.683158,2021-06-18 14:00:07.170079,458--3--2021-06-18 14:00:07.170079,2021-10-04 15:54:12.020790
3,458,1621634402930,4,-23.518139,-46.618745,2021-05-21 19:00:08.924,1621634408924,2021-05-27 09:30:19.683158,2021-06-18 14:00:07.170079,458--4--2021-06-18 14:00:07.170079,2021-10-04 15:54:12.020790
4,458,1621634402930,5,-23.518096,-46.619920,2021-05-21 19:00:13.916,1621634413916,2021-05-27 09:30:19.683158,2021-06-18 14:00:07.170079,458--5--2021-06-18 14:00:07.170079,2021-10-04 15:54:12.020790
